# Starbucks

<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

## My Observations:

Since I wan't to maximize both `IRR` and `NIR`, I need to minimize the **Type 1 Errors**, that is, sending the least amount of promotions to people that won't buy the product. The endgoal is to build a classification model to decide who will receive the information.

## Importing Libs and Reading Data

In [ ]:
# importing libs
from test_results import test_results, score # scripts for testing
from itertools import combinations
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn import svm
from imblearn.over_sampling import SMOTE

%matplotlib inline

In [ ]:
# reading data
df = pd.read_csv('training.csv')
df.head()

In [ ]:
df.info()

**Observations**
- There aren't any nulls in the data (that's good)
- The only object type column is `Promotion` (going to change that to binary)

## Fixing Data

In [ ]:
# fixing the promotion column
df.Promotion.value_counts()

Ok, pretty balanced proportions

In [ ]:
# applying lambda function to turn it into binary
df.Promotion = df.Promotion.apply(lambda x: 0 if x=='No' else 1)
df.Promotion.value_counts()

Good, Let's now explore the data

## EDA

First, let's create different dataframes for each group and explore them individually

In [ ]:
# customers who got the promotion and bought the product (promotion worked)
exp_yes = df.query("Promotion == 1 and purchase == 1")
exp_yes.head()

In [ ]:
# customers who got the promotion and didn't buy the product (type 1 errors)
exp_no = df.query("Promotion == 1 and purchase == 0")
exp_no.head()

In [ ]:
# customers that didn't get the promotion and bought the product (type 2 errors)
con_yes = df.query("Promotion == 0 and purchase == 1")
con_yes.head()

In [ ]:
# customers that didn't get the promotion and didn't buy the product
con_no = df.query("Promotion == 0 and purchase == 0")
con_no.head()

Ok, now that the dataframes are created let's explore and discover how was starbuck managing it's promotions

### 1: Characteristics of customers 

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,10))

sns.countplot(data=exp_yes, x='V1', ax=ax[0,0],edgecolor='black', palette='Set1')
ax[0,0].set_title('Got promotion and Bought')

sns.countplot(data=exp_no, x='V1', ax=ax[0,1],edgecolor='black', palette='Set1')
ax[0,1].set_title('Got promotion and didn\'t Buy')

sns.countplot(data=con_yes, x='V1', ax=ax[1,0],edgecolor='black', palette='Set1')
ax[1,0].set_title('Didn\'t get promotion and Bought')
ax[1,0].set_ylim([0, 300])

sns.countplot(data=con_no, x='V1', ax=ax[1,1],edgecolor='black', palette='Set1')
ax[1,1].set_title('Didn\'t get promotion and didn\'t Buy')
    
plt.show()

**Observations**
- Not much to see here, only that the people who bought tend to have a significant difference between values 1 and 2

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,10))

sns.histplot(data=exp_yes, x='V2', ax=ax[0,0], kde=True)
ax[0,0].set_title('Got promotion and Bought')
ax[0,0].set_xlabel(f'mean:{round(exp_yes.V2.mean(),2)}, std:{round(exp_yes.V2.std(),2)}')

sns.histplot(data=exp_no, x='V2', ax=ax[0,1], kde=True)
ax[0,1].set_title('Got promotion and didn\'t Buy')
ax[0,1].set_xlabel(f'mean:{round(exp_no.V2.mean(),2)}, std:{round(exp_no.V2.std(),2)}')

sns.histplot(data=con_yes, x='V2', ax=ax[1,0], kde=True)
ax[1,0].set_title('Didn\'t get promotion and Bought')
ax[1,0].set_xlabel(f'mean:{round(con_yes.V2.mean(),2)}, std:{round(con_yes.V2.std(),2)}')

sns.histplot(data=con_no, x='V2', ax=ax[1,1], kde=True)
ax[1,1].set_title(f'Didn\'t get promotion and didn\'t Buy')
ax[1,1].set_xlabel(f'mean:{round(con_no.V2.mean(),2)}, std:{round(con_no.V2.std(),2)}')
    
plt.show()

**Observations**
- Basically all of the groups follow the same normal distribution

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,10))

sns.histplot(data=exp_yes, x='V3', ax=ax[0,0], kde=True, bins=20)
ax[0,0].set_title('Got promotion and Bought')
ax[0,0].set_xlabel(f'mean:{round(exp_yes.V3.mean(),2)}, std:{round(exp_yes.V3.std(),2)}')

sns.histplot(data=exp_no, x='V3', ax=ax[0,1], kde=True, bins=20)
ax[0,1].set_title('Got promotion and didn\'t Buy')
ax[0,1].set_xlabel(f'mean:{round(exp_no.V3.mean(),2)}, std:{round(exp_no.V3.std(),2)}')

sns.histplot(data=con_yes, x='V3', ax=ax[1,0], kde=True, bins=20)
ax[1,0].set_title('Didn\'t get promotion and Bought')
ax[1,0].set_xlabel(f'mean:{round(con_yes.V3.mean(),2)}, std:{round(con_yes.V3.std(),2)}')

sns.histplot(data=con_no, x='V3', ax=ax[1,1], kde=True, bins=20)
ax[1,1].set_title(f'Didn\'t get promotion and didn\'t Buy')
ax[1,1].set_xlabel(f'mean:{round(con_no.V3.mean(),2)}, std:{round(con_no.V3.std(),2)}')
    
plt.show()

**Observations**
- Customers who didn't buy the product are basically homogeneous with this feature and have the std as 1
- Customers who bought the product tend to have different results

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,10))

sns.countplot(data=exp_yes, x='V4', ax=ax[0,0],edgecolor='black', palette='Set1')
ax[0,0].set_title('Got promotion and Bought')

sns.countplot(data=exp_no, x='V4', ax=ax[0,1],edgecolor='black', palette='Set1')
ax[0,1].set_title('Got promotion and didn\'t Buy')

sns.countplot(data=con_yes, x='V4', ax=ax[1,0],edgecolor='black', palette='Set1')
ax[1,0].set_title('Didn\'t get promotion and Bought')
ax[1,0].set_ylim([0, 300])

sns.countplot(data=con_no, x='V4', ax=ax[1,1],edgecolor='black', palette='Set1')
ax[1,1].set_title('Didn\'t get promotion and didn\'t Buy')
    
plt.show()

**Observations**
- Not much to see here, all groups are basically the same

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,10))

sns.countplot(data=exp_yes, x='V5', ax=ax[0,0],edgecolor='black', palette='Set1')
ax[0,0].set_title('Got promotion and Bought')

sns.countplot(data=exp_no, x='V5', ax=ax[0,1],edgecolor='black', palette='Set1')
ax[0,1].set_title('Got promotion and didn\'t Buy')

sns.countplot(data=con_yes, x='V5', ax=ax[1,0],edgecolor='black', palette='Set1')
ax[1,0].set_title('Didn\'t get promotion and Bought')
ax[1,0].set_ylim([0, 300])

sns.countplot(data=con_no, x='V5', ax=ax[1,1],edgecolor='black', palette='Set1')
ax[1,1].set_title('Didn\'t get promotion and didn\'t Buy')
    
plt.show()

**Observations**
- Except for the spike in category 3 for people who bought it, they all look the same

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,10))

sns.countplot(data=exp_yes, x='V6', ax=ax[0,0],edgecolor='black', palette='Set1')
ax[0,0].set_title('Got promotion and Bought')

sns.countplot(data=exp_no, x='V6', ax=ax[0,1],edgecolor='black', palette='Set1')
ax[0,1].set_title('Got promotion and didn\'t Buy')

sns.countplot(data=con_yes, x='V6', ax=ax[1,0],edgecolor='black', palette='Set1')
ax[1,0].set_title('Didn\'t get promotion and Bought')
ax[1,0].set_ylim([0, 300])

sns.countplot(data=con_no, x='V6', ax=ax[1,1],edgecolor='black', palette='Set1')
ax[1,1].set_title('Didn\'t get promotion and didn\'t Buy')
    
plt.show()

**Observations**
- Again, no much information from this chart

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,10))

sns.countplot(data=exp_yes, x='V7', ax=ax[0,0],edgecolor='black', palette='Set1')
ax[0,0].set_title('Got promotion and Bought')

sns.countplot(data=exp_no, x='V7', ax=ax[0,1],edgecolor='black', palette='Set1')
ax[0,1].set_title('Got promotion and didn\'t Buy')

sns.countplot(data=con_yes, x='V7', ax=ax[1,0],edgecolor='black', palette='Set1')
ax[1,0].set_title('Didn\'t get promotion and Bought')
ax[1,0].set_ylim([0, 300])

sns.countplot(data=con_no, x='V7', ax=ax[1,1],edgecolor='black', palette='Set1')
ax[1,1].set_title('Didn\'t get promotion and didn\'t Buy')
    
plt.show()

**Observations**
- Again, no much information from this chart

## Checking for Feature Correlations

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']].corr(), annot=True, fmt='.4f')
plt.show()

Ok, features are highly uncorrelated. That's a very big problem.

## Modeling

### 1. One-Hot Encoding Categorical Features

In [ ]:
df_dummies = pd.get_dummies(df, columns=['V1', 'V4', 'V5', 'V6', 'V7'])
df_dummies.head()

### 2. Dividing dataset into training and testing splits

In [ ]:
x = df_dummies.iloc[:, 3:]
y = df_dummies.purchase

sm = SMOTE(random_state=42)
x, y = sm.fit_resample(x, y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=42)

print(x_train.shape)
print(x_test.shape)

### 3. Creating Pipeline

In [ ]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LogisticRegression())
])

param_grid = {'model__C': np.logspace(-4, 4, 20),
              'model__solver': ['liblinear'],
              'model__penalty': ['l1', 'l2'],
              'model__max_iter': [500]}

clf = GridSearchCV(pipe, param_grid)

### 4. Fitting pipeline to training data

In [ ]:
best_clf = clf.fit(x_train, y_train)

### 5. Testing and Evaluating

In [ ]:
yhat = best_clf.predict(x_test)
print(classification_report(y_test, yhat, zero_division=0))
sns.heatmap(confusion_matrix(y_test, yhat), annot=True);